In [ ]:
%matplotlib ipympl


In [ ]:
import sep
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../imaging")
from convenience_functions import show_image

In [ ]:
import fitsio

In [ ]:
    from matplotlib.patches import Ellipse


In [ ]:
from astropy.nddata import CCDData


In [ ]:
img = CCDData.read("../imaging/OB0004/reduced/object/0003882231-20230709-OSIRIS-OsirisBroadBandImage.fits")

In [ ]:
assert img_std.header["filter2"] == img.header["filter2"]

In [ ]:
show_image(img, figsize=(10, 10), log=False, dpi=50)

In [ ]:
bkg = sep.Background(swap_byteorder(img.data))

In [ ]:
show_image(bkg.back(), dpi=50)

In [ ]:
img_nobkg = swap_byteorder(img.data) - bkg


In [ ]:
show_image(img_nobkg)

In [ ]:
objects = sep.extract(img_nobkg, 2, err=bkg.globalrms)


In [ ]:
plot_objects(objects, img_skysub)


In [ ]:
fluxes = []
fluxerrs = []
flags = []

for r in [5, 10, 20, 50]:
    flux, fluxerr, flag = sep.sum_circle(img_nobkg, objects['x'], objects['y'],
                                         r, err=bkg.globalrms, gain=1.0)

    fluxes.append(flux)
    fluxerrs.append(fluxerr)
    flags.append(flag)

In [ ]:
mags = [to_mag(flux) for flux in fluxes]

In [ ]:
plt.figure()
plt.scatter(mags[2], mags[2] - mags[3])

In [ ]:
plt.figure()
plt.scatter(mags[2], mags[2] - mags[3])

In [ ]:
plt.figure()
plt.scatter(mags[1], mags[0] - mags[1], alpha=0.1)

In [ ]:

plt.figure()
plt.hist(mags, bins=20)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))


# Gaia crossmatch

In [ ]:
import astroquery
import astropy.units as u
from astropy import coordinates

In [ ]:
coord0 = coordinates.SkyCoord(img.header["radeg"], img.header["decdeg"], unit=u.deg)

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
Gaia.ROW_LIMIT = 10_000
gaia_sources = Gaia.query_object(coordinate=coord0, radius=30*u.arcmin, verbose=True)

In [ ]:
gaia_sources

In [ ]:
img_stacked = CCDData.read("../imaging/OB0004_sausero/reduced/GTC73-23A_0004_Sloan_g_ast_SKY.fits")

In [ ]:
coords = img_stacked.wcs.pixel_to_world(objects["x"], objects["y"])

In [ ]:
coords_bright = coords[mags[-1] < 22]
ra0, dec0 = img.header["radeg"], img.header["decdeg"]

In [ ]:
coords_gaia = coordinates.SkyCoord(gaia_sources["ra"], gaia_sources["dec"], unit=u.degree)

In [ ]:
xmatch_indicies, xmatch_sep, xmatch_physical = coords_bright.match_to_catalog_sky(coords_gaia)

In [ ]:
has_xmatch = xmatch_sep < 1*u.arcsec

In [ ]:
plt.figure()
plt.scatter(gaia_sources["ra"] - ra0, gaia_sources["dec"] - dec0)
plt.scatter(coords_bright.ra / u.degree - ra0, coords_bright.dec / u.degree - dec0, s=5)
plt.xlim(-0.1, 0.1)
plt.ylim(-0.1, 0.1)

In [ ]:
gaia_xmatch = gaia_sources[[xmatch_indicies[has_xmatch]]]
osiris_xmatch = coords_bright[has_xmatch]

In [ ]:
plt.figure()
plt.scatter(mags[2][mags[3] < 22][has_xmatch], gaia_xmatch["phot_bp_mean_mag"])
plt.xlabel("g")
plt.ylabel("gaia G")

In [ ]:
ra0, dec0

In [ ]:
plt.figure()
plt.scatter(gaia_sources[xmatch_indicies[has_xmatch]]["ra"] - ra0, gaia_sources[xmatch_indicies[has_xmatch]]["dec"] - dec0)
plt.scatter(coords_bright[has_xmatch].ra / u.degree - ra0, coords_bright[has_xmatch].dec / u.degree - dec0, s=5)
plt.xlim(-0.1, 0.1)
plt.ylim(-0.1, 0.1)

### PanStarrs

In [ ]:
from astropy.table import Table

In [ ]:
panstarrs = Table.read("../survey_data/yasone2_panstarrs.fits")

In [ ]:

plt.figure()
plt.scatter(panstarrs["ra"], panstarrs["dec"])

In [ ]:
?coords_bright.match_to_catalog_sky

In [ ]:
coords_panstarrs.shape

In [ ]:
panstarrs["ra"].reshape(-1)

In [ ]:
coords_panstarrs = coordinates.SkyCoord(panstarrs["ra"].reshape(-1), panstarrs["dec"].reshape(-1), unit=u.degree)
xmatch_indicies_panstarrs, xmatch_sep_panstarrs, xmatch_physical_panstarrs = coords.match_to_catalog_sky(coords_panstarrs)
has_xmatch_panstarrs = xmatch_sep_panstarrs < 1*u.arcsec

In [ ]:
panstarrs["ra"].transpose()

In [ ]:
coords_panstarrs

In [ ]:
panstarrs_xmatch = panstarrs[xmatch_indicies_panstarrs[has_xmatch_panstarrs]]
osiris_x_panstarrs = coords[has_xmatch_panstarrs]

In [ ]:
osiris_x_panstarrs

In [ ]:
plt.figure()
plt.scatter(panstarrs_xmatch["ra"] - ra0, panstarrs_xmatch["dec"] - dec0)
plt.scatter(osiris_x_panstarrs.ra/u.degree - ra0, osiris_x_panstarrs.dec/u.degree - dec0, s=5)
plt.xlim(-0.1, 0.1)
plt.ylim(-0.1, 0.1)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)
plt.sca(axs[0])
plt.scatter(mags[1][has_xmatch_panstarrs], panstarrs_xmatch["gMeanPSFMag"], s=2)
plt.xlim(26, 10)
plt.ylim(26, 10)
plt.plot([26, 10], [26, 10], color="black")
plt.ylabel("magnitude (panstarrs)")

plt.sca(axs[1])

plt.scatter(mags[-3][has_xmatch_panstarrs], panstarrs_xmatch["gMeanPSFMag"].reshape(-1) - mags[-3][has_xmatch_panstarrs], s=2)
plt.ylim(-1, 1)
plt.axhline(0, color="black")

plt.xlabel("magnitude (me)")
